# 上下文处理器
- 它是可以返回一些数据，在全局模板中都可以使用。比如登录用户的信息，在很多页面中都需要使用，那么我们可以放在上下文处理器中，就没有必要在每个视图函数中返回这个对象
- 在settings.TEMPLATES.OPTIONS.context_processors中，有许多内置的上下文处理器，这些上下文处理器的作用如下：
    - 1.django.template.context_processors.debug：增加一个debug和sql_queries变量，在模板中可以通过他来查到到一些数据库查询信息
        - 想要使用debug，先在settings.py文件中设置INTERANL_IPS =['127.0.0.1'],然后执行SQL操作，在模板文件中使用{{debug}} {{sql_queries}}即可获取查询记录
    - 2.django.template.context_processors.request：增加一个request变量，这个request变量也就是在视图函数的第一个参数。这个上下文管理器可以使得在模板文件中也可以使用request，如{{request}}
    - 3.django.contrib.auth.context_processors.auth：django有内置的用户系统，这个上下文处理器会增加一个user对象
    - 4.django.contrib.messages.context_processors.messages:增加一个messages变量。也就是出错时的那个错误信息提示
        - 导入：from django.contrib import messages
        - messages.add_message(request, 错误级别，信息提示)
        - 比如：messages.add_message(request,messages.INFO, "用户名错误")或者简写方式:messages.info(request, "用户名错误")
        - 然后在模板中就可以直接遍历使用messages了
                {% for message in messages %}
                    {{ message}}
                {% endfor %}
    - 5.django.template.context_processors.media:在模板中读取MEDIA_URL.比如想要在模板中使用上传的文件，那么这时候就需要使用settings.py中设置的MEDIA_URL来拼接url。
        - 1.先在项目下新建medias文件夹
        - 2.在settings中设置:(medias是上传的文件的保存地址)
                MEDIA_ROOT = os.path.join(BASE_DIR, 'medias')
                MEDIA_URL = '/medias/'
        - 3.在TEMPLATES/OPTIONS下加入：django.template.context_processors.media
        - 4.在模板文件中使用上传的文件
            - 比如：img src="{{MEDIA_URL}} {{ user.avatar}}"
    - 6.django.template.context_processors.static:在模板中使用STATIC_URL
    - 7.django.template.context_processors.csrf:在模板中可以使用csrf_token变量来生成一个csrf_token
        - 如果没有注释它：'django.middleware.csrf.CsrfViewMiddleware',，就需要在提交表单的时候进行提交csrftoken
        - 1.首先在settings.py文件中TEMPLATES/OPTIONS下加入django.template.context_processors.csrf
        - 2.然后在模板中使用input标签进行提交：input type='hidden' name="csrfmiddlewaretoken" value={{ csrf_token }}
        - 如果不想那么麻烦，直接一步到位，在模板中使用{% csrf_token %}

# 自定义上下文处理器
- 有时候我们想要返回自己的数据，这时可以自定义上下文处理器，步骤如下：
    - 1.根据这个上下文归属于哪个APP，在这个APP中创建一个文件专门用来存储上下文处理器。比如context_processors.py，或者专门创建一个python包，用来存储所有的上下文处理器。定义好上下文处理器后，需要在TEMPLATES来注册该上下文处理器：APP名字.上下文py文件.定义的函数
        - 示例：
                from .models import UserModel

                # 上下文处理器需要在TEMPLATES中进行设置
                def UserInfo(request):
                    user_id = request.session.get("user_id")
                    context = {}
                    if user_id:
                        try:
                            user = UserModel.objects.get(pk=user_id)
                            context['user_info'] = user
                        except:
                            pass
                    return context
                    
                在settings.py中注册
                'OPTIONS': {
                'context_processors': [
                    'django.template.context_processors.debug',
                    'django.template.context_processors.request',
                    'django.contrib.auth.context_processors.auth',
                    'django.contrib.messages.context_processors.messages',
                    'one_app.context_processor.UserInfo', #APP名字.上下文py文件.定义的函数
                ],

    - 2.在定义好的上下文处理器文件中，定义一个函数，这个函数只有一个request参数。这个函数中处理完自己的逻辑后，把需要的数据以字典的形式返回给模板。如果不需要返回任何数据，那么也必须返回一个空的字典。